# Test

In [1]:
import os
import sys
import pandas as pd

In [2]:
sys.path.insert(0, os.path.abspath(".."))

In [3]:
from acro import ACRO

### Instantiate ACRO

In [4]:
acro = ACRO()

DEBUG:acro:path: /home/unknown/work/current/ACRO/ACRO/acro/default.yaml
DEBUG:acro:config: {'output_template_file': 'ACRO output template v01b.xlsm', 'safe_SDC_set': 'Default', 'safe_tests': 'threshold nk pratio maxmin', 'safe_threshold': 10, 'safe_dof_threshold': 10, 'safe_nk_n': 2, 'safe_nk_k': 0.9, 'safe_pratio_p': 0.1, 'safe_SDC_variations': 'CIS ESS', 'safe_thresholdCIS': 60, 'safe_nk_nCIS': 5, 'safe_nk_kCIS': 0.5, 'safe_testsCIS': 'threshold nk pratio', 'safe_thresholdESS': 15, 'safe_dof_thresholdESS': 10, 'safe_nk_nESS': 2, 'safe_nk_kESS': 0.9, 'safe_pratio_pESS': 0.15, 'safe_testsESS': 'nk pratio'}


### Load test data

In [5]:
path = os.path.join("../data", "test_data.dta")
df = pd.read_stata(path)
df.head()

,charity,grant_type,index,year,inc_activity,inc_grants,inc_donations,inc_other,inc_total,total_costs,...,sh_staff_grants_given,sh_assets_grants_given,sh_income_balance,sh_staff_balance,sh_assets_balance,sh_income_assets,sh_staff_assets,sh_income_staff_costs,sh_assets_staff_costs,wgt
0,4Children,R,1.0,2011,2880902.0,9603182.0,91404.0,310947.0,12886435.0,12127472.0,...,NaN,NaN,0.072636,0.135971,0.767809,0.094602,0.177090,0.534203,5.646843,1.0
1,4Children,R,1.0,2014,6810520.0,18768904.0,58002.0,401879.0,26039304.0,25493796.0,...,NaN,NaN,0.057641,0.089150,1.001396,0.057560,0.089026,0.646561,11.232729,1.0
2,4Children,R,1.0,2015,7199403.0,21638036.0,132191.0,512654.0,29482284.0,32290108.0,...,NaN,NaN,-0.049619,-0.079828,-0.620210,0.080004,0.128711,0.621583,7.769365,1.0
3,4Children,R,1.0,2013,5573013.0,15194731.0,228844.0,267156.0,21263744.0,20989048.0,...,NaN,NaN,0.045740,0.068251,1.008259,0.045365,0.067692,0.670166,14.772749,1.0
4,4Children,R,1.0,2010,2056816.0,7335103.0,110256.0,424628.0,9926803.0,9769816.0,...,NaN,NaN,0.057696,0.122532,0.567539,0.101660,0.215901,0.470862,4.631749,1.0


## Create a simple linear regression in three ways
 and print the results object returned and how to access the degrees of freedom
 
 I will start with inc_total vs inc_grants

### First sklearn


In [6]:
from sklearn.linear_model import LinearRegression

df.dropna(inplace=True)
y = df["inc_total"].values.reshape(-1, 1)  # values converts it into a numpy array
x = df["inc_grants"].values.reshape(
    -1, 1
)  # -1 means that calculate the dimension of rows, but have 1 column
linear_regressor = LinearRegression()  # create object for the class
res = linear_regressor.fit(x, y)  # perform linear regression
y_pred = linear_regressor.predict(x)  # make predictions

In [7]:
print(f"intercept {linear_regressor.intercept_}, coeffecients {linear_regressor.coef_}")

intercept [29906766.], coeffecients [[1.8979715]]


### now a linear regression with two input variables

In [8]:
x2 = df[["inc_grants", "inc_activity"]].values.reshape(
    -1, 2
)  # -1 means that calculate the dimension of rows, but have 1 column
bilinear_regressor = LinearRegression()  # create object for the class
res2 = bilinear_regressor.fit(x2, y)  # perform linear regression
y_pred2 = bilinear_regressor.predict(x2)  # make predictions

print(
    f"intercept {bilinear_regressor.intercept_}, coeffecients {bilinear_regressor.coef_}"
)

intercept [12081892.], coeffecients [[1.7231662 1.2810115]]


### for completeness a linear regression with three input variables but no intercept

In [9]:
x3 = df[["inc_grants", "inc_activity", "inc_donations"]].values.reshape(
    -1, 3
)  # -1 means that calculate the dimension of rows, but have 1 column
trilinear_regressor = LinearRegression(
    fit_intercept=False
)  # create object for the class
res3 = trilinear_regressor.fit(x3, y)  # perform linear regression
y_pred3 = trilinear_regressor.predict(x3)  # make predictions


print(
    f"intercept {trilinear_regressor.intercept_}, coeffecients {trilinear_regressor.coef_}"
)

intercept 0.0, coeffecients [[1.0145859 1.0113665 1.0529852]]


### Function to calculate the degrees of freedom of a model 
(number of parameters that can vary) will depend on whether an intercept is allowed.

In [10]:
type(linear_regressor)

sklearn.linear_model._base.LinearRegression

In [11]:
import sklearn


def get_dof(model) -> int:
    if isinstance(model, sklearn.linear_model._base.LinearRegression):
        dof = model.n_features_in_
        if model.fit_intercept:
            dof = dof + 1
    return dof

In [12]:
print(
    "Function reports that the Dof for:\n"
    f"\t\tsimple linear model is {get_dof(linear_regressor)}\n"
    f"\t\tmodel with two variables and intercept is {get_dof(bilinear_regressor)}\n"
    f"\t\tand for model with 3 variables and no intercept is {get_dof(trilinear_regressor)}."
)

Function reports that the Dof for:
		simple linear model is 2
		model with two variables and intercept is 3
		and for model with 3 variables and no intercept is 3.


## Polynomial regression using sklearn models by transforming data
### starting with a simple single input value

In [13]:
from sklearn.preprocessing import PolynomialFeatures

# get a clean version of the data
path = os.path.join("../data", "test_data.dta")
df = pd.read_stata(path)
df.dropna(inplace=True)
y = df["inc_total"].values.reshape(-1, 1)  # values converts it into a numpy array
x = df["inc_grants"].values.reshape(
    -1, 1
)  # -1 means that calculate the dimension of rows, but have 1 column


# one liner to make new data rather than explicitly calling constructor,fit, then transform
degree = 2
x_ = PolynomialFeatures(degree=degree, include_bias=False).fit_transform(x)

print(f"so to fit a polynomial of degree {degree}, data now has shape {x_.shape}")
print(x_[0:5, :])

# fit the data
polymodel = LinearRegression().fit(x_, y)
print(f"intercept {polymodel.intercept_}, coefficients {polymodel.coef_}")
print(f"and our function reports the degrees of freedom as {get_dof(polymodel)}")

so to fit a polynomial of degree 2, data now has shape (640, 2)
[[3.000e+04 9.000e+08]
 [0.000e+00 0.000e+00]
 [0.000e+00 0.000e+00]
 [0.000e+00 0.000e+00]
 [4.500e+04 2.025e+09]]
intercept [37651692.], coefficients [[4.0277274e-17 7.8147604e-09]]
and our function reports the degrees of freedom as 3


### and now multiple input values in the same framework
e.g. with 2 inputs and polynomial of degree 2, the estiamted regression function is 
𝑓(𝑥₁, 𝑥₂) = 𝑏₀ + 𝑏₁𝑥₁ + 𝑏₂𝑥₂ + 𝑏₃𝑥₁² + 𝑏₄𝑥₁𝑥₂ + 𝑏₅𝑥₂².

In [14]:
x2 = df[["inc_grants", "inc_activity"]].values.reshape(-1, 2)

degree = 2
x2_ = PolynomialFeatures(degree=degree, include_bias=False).fit_transform(x2)

print(f"so to fit a polynomial of degree {degree}, data now has shape {x2_.shape}")
print("first five rows are:")
print(x2_[0:5, :])

# fit the data
polymodel2 = LinearRegression().fit(x2_, y)
print(f"intercept {polymodel2.intercept_}, coefficients {polymodel2.coef_}")
print(f"and our function reports the degrees of freedom as {get_dof(polymodel2)}")

so to fit a polynomial of degree 2, data now has shape (640, 5)
first five rows are:
[[3.0000000e+04 8.1703200e+05 9.0000000e+08 2.4510960e+10 6.6754131e+11]
 [0.0000000e+00 5.2915700e+05 0.0000000e+00 0.0000000e+00 2.8000711e+11]
 [0.0000000e+00 4.2903900e+05 0.0000000e+00 0.0000000e+00 1.8407447e+11]
 [0.0000000e+00 4.8266800e+05 0.0000000e+00 0.0000000e+00 2.3296840e+11]
 [4.5000000e+04 7.3457300e+05 2.0249999e+09 3.3055785e+10 5.3959750e+11]]
intercept [23210134.], coefficients [[ 4.8942315e-16 -5.4178884e-14  3.0109391e-09  6.4171907e-08
   1.9352653e-09]]
and our function reports the degrees of freedom as 6


### what if they provide intercept in data rather than in call to model

In [15]:
x3_ = PolynomialFeatures(degree=degree).fit_transform(x2)

print(
    f"so to fit a polynomial of degree {degree}",
    "but including an intercept," f"data now has shape {x3_.shape}",
)
print("first five rows are:")
print(f"{x3_[0:5,:]:}")

# fit the data
polymodel3 = LinearRegression(fit_intercept=False).fit(x3_, y)
print(f"intercept {polymodel3.intercept_}, coefficients {polymodel3.coef_}")
print(f"and our function reports the degrees of freedom as {get_dof(polymodel3)}")

so to fit a polynomial of degree 2 but including an intercept,data now has shape (640, 6)
first five rows are:
[[1.0000000e+00 3.0000000e+04 8.1703200e+05 9.0000000e+08 2.4510960e+10
  6.6754131e+11]
 [1.0000000e+00 0.0000000e+00 5.2915700e+05 0.0000000e+00 0.0000000e+00
  2.8000711e+11]
 [1.0000000e+00 0.0000000e+00 4.2903900e+05 0.0000000e+00 0.0000000e+00
  1.8407447e+11]
 [1.0000000e+00 0.0000000e+00 4.8266800e+05 0.0000000e+00 0.0000000e+00
  2.3296840e+11]
 [1.0000000e+00 4.5000000e+04 7.3457300e+05 2.0249999e+09 3.3055785e+10
  5.3959750e+11]]
intercept 0.0, coefficients [[ 1.1876045e-23 -1.4210855e-14  1.8235969e-15  3.4045611e-09
   6.8190062e-08  2.0831159e-09]]
and our function reports the degrees of freedom as 6


In [16]:
print(
    "if they include the intercept when transforming,"
    " making a first column of 1s in the data as above,"
    " but then also include the intercept in the model"
    " we get this:"
)

# fit the data

polymodel4 = LinearRegression(fit_intercept=True).fit(x3_, y)
print(f"intercept {polymodel4.intercept_}, coefficients {polymodel4.coef_}")
print(f"and our function reports the degrees of freedom as {get_dof(polymodel4)}")

if they include the intercept when transforming, making a first column of 1s in the data as above, but then also include the intercept in the model we get this:
intercept [23210144.], coefficients [[ 0.0000000e+00 -1.3322676e-14  4.1602384e-17  3.0109304e-09
   6.4171935e-08  1.9352613e-09]]
and our function reports the degrees of freedom as 7


### <span style="color:red">Question: What should we do in this case?   
- We are going to release less with lower degrees of freedom.
- So maybe we should say if intercept ==0.00000 we do not count it?   
  i.e. 
    >         if model.fit_intercept :
    >              dof = dof +1
  becomes
    >         if model.fit_intercept and not np.isclose(model.intercept_ ,0.0):
    >               dof = dof +1
 - I've used two clauses in case the intee4cept is not always nicely initialsied to zero when unused 

## <span style="color:red">Answer: See email reponse from Paul White</span>
- The code above calculates the model DoF:P
- The value that we compare to the threshold is *N -P -1*
  - also known as the 'residual dof'
 
- Paul has agreed with the suggestion that in practice we should use *S* rather than *P*, 
  where *S* is the number of co-efficients which are statistically significantly different to zero  
  taking into account the number of observations. 
  - OLS returns the signmificabnce/t-tst vals for each co-efficient in its results object
  - we would have to compute this (2-sided t-test?) if we used the sklearn version.
  

# TO-DO
1. Repeat this process for different ways of getting a linear regression i.e. via scipy.stats
   - in some cases we may be able to get at the dof directly if the fit process returns a results object
   - or we could consider getting at it direcrtly from the combination of the number of colu,mns in x and whether an intercept is allowed
   - either way, build up the get_dof() function
   
2. Create a linear_regression() mewthod within the aCRO class that used statsmodel version ut then checks for the DoF against a supplied threshold value (use 6)

# ACRO

### Pass

In [17]:
y = df["inc_total"].values.reshape(-1, 1)
x = df["inc_grants"].values.reshape(-1, 1)
# ACRO OLS - auto fits
results = acro.ols(y, x)
results.summary()

DEBUG:acro:ols() outcome: pass; dof=639.0 >= 10
DEBUG:acro:add_output(): output_0


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.344
Model:                            OLS   Adj. R-squared (uncentered):              0.343
Method:                 Least Squares   F-statistic:                              334.8
Date:                Mon, 26 Sep 2022   Prob (F-statistic):                    1.88e-60
Time:                        16:36:18   Log-Likelihood:                         -12585.
No. Observations:                 640   AIC:                                  2.517e+04
Df Residuals:                     639   BIC:                                  2.518e+04
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             2.1779      0.119     18.297      0.000       1.944       2.412
==============================================================================
Omnibus:                      450.906   Durbin-Watson:                   0.428
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5423.895
Skew:                           3.097   Prob(JB):                         0.00
Kurtosis:                      15.846   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Fail

In [18]:
import numpy as np

y = np.linspace(0, 1, 6)
x = np.linspace(0, 1, 6)
# ACRO OLS - auto fits
results = acro.ols(y, x)
results.summary()

/home/unknown/work/current/ACRO/ACRO/acro/acro.py:618: UserWarning: Unsafe OLS: fail; dof=5.0 < 10
  warnings.warn(f"Unsafe OLS: {outcome}")
DEBUG:acro:ols() outcome: fail; dof=5.0 < 10
DEBUG:acro:add_output(): output_1
/home/unknown/.local/lib/python3.10/site-packages/statsmodels/stats/stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 6 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "
/home/unknown/.local/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:1825: RuntimeWarning: divide by zero encountered in double_scalars
  return self.mse_model/self.mse_resid
/home/unknown/.local/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:924: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
/home/unknown/.local/lib/python3.10/site-packages/statsmodels/stats/stattools.py:50: RuntimeWarning: invalid value encountere

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   1.000
Model:                            OLS   Adj. R-squared (uncentered):              1.000
Method:                 Least Squares   F-statistic:                                inf
Date:                Mon, 26 Sep 2022   Prob (F-statistic):                        0.00
Time:                        16:36:18   Log-Likelihood:                             inf
No. Observations:                   6   AIC:                                       -inf
Df Residuals:                       5   BIC:                                       -inf
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             1.0000          0        inf      0.000       1.000       1.000
==============================================================================
Omnibus:                          nan   Durbin-Watson:                     nan
Prob(Omnibus):                    nan   Jarque-Bera (JB):                  nan
Skew:                             nan   Prob(JB):                          nan
Kurtosis:                         nan   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Outputs

In [19]:
outputs = acro.get_outputs()
outputs

DEBUG:acro:get_outputs()


{'output_0': {'command': 'results = acro.ols(y, x)',
  'summary': 'pass',
  'outcome': 'pass; dof=639.0 >= 10',
  'output': {}},
 'output_1': {'command': 'results = acro.ols(y, x)',
  'summary': 'fail',
  'outcome': 'fail; dof=5.0 < 10',
  'output': {}}}